<a href="https://colab.research.google.com/github/AHANAAF/CCFD-ML/blob/main/Mpox_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import files

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Load the dataset (use the file you uploaded, e.g., 'mpox_spread_data_30k.csv')
df = pd.read_csv('mpox_30kv1.csv')

# Step 3: Check the structure of the data (this will display the first few rows)
print(df.head())

# Step 4: Features (X) and target (y) selection
# Ensure that the columns exist in your dataset
X = df[['Latitude', 'Longitude', 'Population_Density', 'Mobility_Score',
        'Social_Contacts', 'Temperature', 'Humidity', 'Vaccination_Rate', 'Distance_to_Case']]
y = df['Status']  # Assuming 'Status' column contains binary 0 or 1 for the outbreak risk

# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Scaling the features
scaler = StandardScaler()

# Fit on training data and transform both train and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Check shapes to ensure everything is split and scaled correctly
print(f"X_train shape: {X_train_scaled.shape}")
print(f"X_test shape: {X_test_scaled.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


Saving mpox_30kv1.csv to mpox_30kv1.csv
    Latitude   Longitude  Population_Density  Mobility_Score  Social_Contacts  \
0 -22.582779   49.732045        22246.656094        0.877159        13.845643   
1  81.128575  -14.654717        26433.056253        0.907172        54.432865   
2  41.758910  167.219469        13895.396261        0.378372        61.923246   
3  17.758527 -101.167758         8675.361950        0.231631        51.262822   
4 -61.916645   31.628310         9565.397231        0.191661        52.242124   

   Temperature   Humidity  Vaccination_Rate  Distance_to_Case  Status  
0   -24.463066  21.409283         35.713649          3.756800       1  
1    16.855740  75.173715         91.594739         17.294752       1  
2    33.909499  29.146152         98.231101         33.323000       0  
3    31.157848  89.630332         96.744697         15.265796       0  
4    36.991697  39.561455         62.827212         28.707923       0  
X_train shape: (24000, 9)
X_test shape: (

In [4]:
from tensorflow.keras import models, layers

# Define the DNN model
def create_dnn_model(input_shape):
    model = models.Sequential()

    # Add an Input layer
    model.add(layers.Input(shape=input_shape))

    # Add Dense layers
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))

    # Output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Create the model
input_shape = (X_train_scaled.shape[1],)  # Number of features
dnn_model = create_dnn_model(input_shape)


In [5]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping with increased patience
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Create a DNN model with batch normalization and regularization
def create_dnn_model(input_shape):
    from tensorflow.keras import models, layers, regularizers
    import tensorflow

    model = models.Sequential()

    # Input layer
    model.add(layers.Dense(128, activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())

    # Hidden layers with batch normalization and dropout
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    # Output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model with a lower learning rate
    model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Define the input shape based on the number of features in the dataset
input_shape = (X_train_scaled.shape[1],)
dnn_model = create_dnn_model(input_shape)

# Train the model with a reasonable epoch count and early stopping
dnn_model.fit(X_train_scaled, y_train,
              epochs=300,
              batch_size=64,
              validation_split=0.2,
              callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_acc = dnn_model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_acc}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.8804 - loss: 3.0127 - val_accuracy: 0.9675 - val_loss: 1.3103
Epoch 2/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9530 - loss: 1.0615 - val_accuracy: 0.9760 - val_loss: 0.5079
Epoch 3/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9490 - loss: 0.4767 - val_accuracy: 0.9812 - val_loss: 0.2480
Epoch 4/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9589 - loss: 0.2606 - val_accuracy: 0.9796 - val_loss: 0.1535
Epoch 5/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9601 - loss: 0.1828 - val_accuracy: 0.9823 - val_loss: 0.1102
Epoch 6/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9669 - loss: 0.1404 - val_accuracy: 0.9829 - val_loss: 0.0889
Epoch 7/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9677 - loss: 0.1271 - val_accuracy: 0.9892 - val_loss: 0.0729
Epoch 8/300
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9743 - loss: 0.1007 - val_acc

In [12]:
# Function to predict risk based on user input
def predict_outbreak_risk(user_latitude, user_longitude, population_density, radius_km, symptoms_score):
    import numpy as np # Import numpy within the function's scope
    # Find nearby cases within the radius
    nearby_cases = df[(df['Latitude'].between(user_latitude - radius_km, user_latitude + radius_km)) &
                      (df['Longitude'].between(user_longitude - radius_km, user_longitude + radius_km))]

    if nearby_cases.empty:
        print("No nearby cases found within the given radius.")
        return None

    avg_population_density = nearby_cases['Population_Density'].mean()
    avg_mobility_score = nearby_cases['Mobility_Score'].mean()
    avg_social_contacts = nearby_cases['Social_Contacts'].mean()
    avg_temperature = nearby_cases['Temperature'].mean()
    avg_humidity = nearby_cases['Humidity'].mean()
    avg_vaccination_rate = nearby_cases['Vaccination_Rate'].mean()
    avg_distance_to_case = nearby_cases['Distance_to_Case'].mean()

    # Input data for prediction
    input_data = np.array([[user_latitude, user_longitude, avg_population_density, avg_mobility_score,
                            avg_social_contacts, avg_temperature, avg_humidity, avg_vaccination_rate,
                            avg_distance_to_case]]) # Remove symptoms_score from input_data

    # Scale input data
    input_data_scaled = scaler.transform(input_data)

    # Predict outbreak risk
    prediction = dnn_model.predict(input_data_scaled)
    return prediction[0][0]

# Example user input
user_lat = float(input("Enter your latitude: "))
user_lon = float(input("Enter your longitude: "))
population_density = float(input("Enter approximate population density: "))
radius_km = float(input("Enter radius in kilometers: "))
symptoms_score = float(input("Enter symptoms score (1-10): "))

# Predict outbreak risk
risk_prediction = predict_outbreak_risk(user_lat, user_lon, population_density, radius_km, symptoms_score)
if risk_prediction is not None:
    print(f"Predicted Risk of Mpox Outbreak: {risk_prediction}")

Enter your latitude: 28.6139
Enter your longitude: 77.2090
Enter approximate population density: 20000
Enter radius in kilometers: 15
Enter symptoms score (1-10): 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Risk of Mpox Outbreak: 0.025546642020344734


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
